# Opioids Project: Data Cleaning

Ra'Kira Nelson and Alexa Fahrer

In [6]:
import pandas as pd

pd.set_option("mode.copy_on_write", True)

## Prescriptions

In [7]:
prescriptions_raw = pd.read_parquet("data/ids590_opioids_by_drug_county_year.parquet")
prescriptions = prescriptions_raw.copy()

In [8]:
prescriptions["mme_conversion_factor"] = (
    prescriptions["mme_conversion_factor"].to_numpy().astype("float64")
)
prescriptions["calc_base_wt_in_gm"] = (
    prescriptions["calc_base_wt_in_gm"].to_numpy().astype("float64")
)
prescriptions["buyer_county"] = prescriptions["buyer_county"].str.upper().str.strip()
prescriptions["buyer_state"] = prescriptions["buyer_state"].str.upper().str.strip()

prescriptions = prescriptions[
    ~prescriptions["buyer_state"].isin(["PR", "VI", "GU", "MP", "AS", "PW"])
]

In [9]:
prescriptions.head(20)

,buyer_state,buyer_county,year,drug_name,mme_conversion_factor,calc_base_wt_in_gm
0,AK,ANCHORAGE,2006,BUPRENORPHINE,30.00,125.154336
1,AK,ANCHORAGE,2006,BUPRENORPHINE,75.00,0.216367
2,AK,ANCHORAGE,2006,CODEINE,0.15,13362.990030
3,AK,ANCHORAGE,2006,DIHYDROCODEINE,0.25,4.272640
4,AK,ANCHORAGE,2006,FENTANYL,100.00,476.051283
5,AK,ANCHORAGE,2006,FENTANYL,130.00,33.141598
6,AK,ANCHORAGE,2006,HYDROCODONE,1.00,21830.672299
7,AK,ANCHORAGE,2006,HYDROMORPHONE,4.00,1351.941549
8,AK,ANCHORAGE,2006,LEVORPHANOL,11.00,1.294480
9,AK,ANCHORAGE,2006,MEPERIDINE,0.10,8142.206720


In [10]:
prescriptions_condensed = prescriptions.groupby(
    ["buyer_state", "buyer_county", "year"], as_index=False, observed=True
)["calc_base_wt_in_gm"].sum()
prescriptions_condensed

,buyer_state,buyer_county,year,calc_base_wt_in_gm
0,AK,ANCHORAGE,2006,143823.375649
1,AK,ANCHORAGE,2007,145660.162020
2,AK,ANCHORAGE,2008,151111.681778
3,AK,ANCHORAGE,2009,154228.894910
4,AK,ANCHORAGE,2010,153320.451041
...,...,...,...,...
42117,WY,WESTON,2015,2725.641288
42118,WY,WESTON,2016,2284.998660
42119,WY,WESTON,2017,1893.343744
42120,WY,WESTON,2018,1701.088712


## FIPS

In [11]:
fips = pd.read_excel("data/US_FIPS_Codes.xls", skiprows=1)

In [12]:
fips["fips"] = fips["FIPS State"].astype(str).str.zfill(2) + fips["FIPS County"].astype(
    str
).str.zfill(3)

us_state_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "District of Columbia": "DC",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
}

fips["state_abbrev"] = fips["State"].map(us_state_abbrev)
fips["County Name"] = fips["County Name"].str.upper().str.strip()
fips["state_abbrev"] = fips["state_abbrev"].str.upper().str.strip()

fips["County Name"] = (
    fips["County Name"]
    .str.upper()
    .str.strip()
    .str.replace(r"^ST[.\s]+", "SAINT ", regex=True)
)

fips

,State,County Name,FIPS State,FIPS County,fips,state_abbrev
0,Alabama,AUTAUGA,1,1,01001,AL
1,Alabama,BALDWIN,1,3,01003,AL
2,Alabama,BARBOUR,1,5,01005,AL
3,Alabama,BIBB,1,7,01007,AL
4,Alabama,BLOUNT,1,9,01009,AL
...,...,...,...,...,...,...
3137,Wyoming,SWEETWATER,56,37,56037,WY
3138,Wyoming,TETON,56,39,56039,WY
3139,Wyoming,UINTA,56,41,56041,WY
3140,Wyoming,WASHAKIE,56,43,56043,WY


In [13]:
# prescriptions_condensed["buyer_county"] = prescriptions_condensed[
#     "buyer_county"
# ].replace(
#     {
#         # Alaska
#         "PETERSBURG": "WRANGELL PETERSBURG",
#         "PRINCE OF WALES HYDER": "PRINCE WALES KETCHIKAN",
#         "SKAGWAY": "SKAGWAY HOONAH ANGOON",
#         "WRANGELL": "WRANGELL PETERSBURG",
#         # Georgia / Illinois
#         "DEKALB": "DE KALB",
#         # Illinois
#         "DUPAGE": "DU PAGE",
#         # Indiana
#         "ST JOSEPH": "SAINT JOSEPH",
#         # Louisiana
#         "ST JOHN THE BAPTIST": "SAINT JOHN THE BAPTIST",
#         # Missouri
#         "SAINTE GENEVIEVE": "STE GENEVIEVE",
#         # Mississippi
#         "DESOTO": "DE SOTO",
#         # Virginia
#         "SALEM": "SALEM CITY",
#     }
# )

In [14]:
prescriptions_fips_merged = pd.merge(
    prescriptions_condensed,
    fips[["state_abbrev", "County Name", "fips"]],
    left_on=["buyer_state", "buyer_county"],
    right_on=["state_abbrev", "County Name"],
    how="left",
    indicator=True,
    validate="m:1",
)

prescriptions_merged = prescriptions_fips_merged.drop(
    columns=["state_abbrev", "County Name", "_merge"]
).copy()

In [15]:
print(prescriptions_fips_merged["_merge"].value_counts())

_merge
both          41945
left_only       177
right_only        0
Name: count, dtype: int64


In [16]:
unmatched = prescriptions_fips_merged[
    prescriptions_fips_merged["_merge"] == "left_only"
]
unmatched[["buyer_state", "buyer_county"]].drop_duplicates().sort_values(
    ["buyer_state", "buyer_county"]
)

,buyer_state,buyer_county
182,AK,PETERSBURG
196,AK,PRINCE OF WALES HYDER
224,AK,SKAGWAY
253,AK,WRANGELL
1873,AR,MONTGOMERY
5734,GA,DEKALB
9527,IL,DEKALB
9569,IL,DUPAGE
11710,IN,ST JOSEPH
15765,LA,ST JOHN THE BAPTIST


In [17]:
prescriptions_merged

,buyer_state,buyer_county,year,calc_base_wt_in_gm,fips
0,AK,ANCHORAGE,2006,143823.375649,02020
1,AK,ANCHORAGE,2007,145660.162020,02020
2,AK,ANCHORAGE,2008,151111.681778,02020
3,AK,ANCHORAGE,2009,154228.894910,02020
4,AK,ANCHORAGE,2010,153320.451041,02020
...,...,...,...,...,...
42117,WY,WESTON,2015,2725.641288,56045
42118,WY,WESTON,2016,2284.998660,56045
42119,WY,WESTON,2017,1893.343744,56045
42120,WY,WESTON,2018,1701.088712,56045


## Deaths

In [18]:
deaths_dfs = {}
for year in range(2003, 2016):
    key = f"deaths_{year}"
    url = (
        "https://media.githubusercontent.com/media/nickeubank/ids540_opioid_data/"
        f"refs/heads/main/vitalstatistics/Underlying%20Cause%20of%20Death%2C%20{year}.txt"
    )

    df = pd.read_csv(url, sep="\t", skipfooter=15, engine="python")
    df = df.drop(columns=["Notes"])
    deaths_dfs[key] = df

deaths = pd.concat(
    [deaths_dfs[f"deaths_{year}"].assign(year=year) for year in range(2006, 2016)],
    ignore_index=True,
)

In [19]:
deaths["Year"] = pd.to_numeric(deaths["Year"], errors="coerce").astype("Int64")
deaths["Deaths"] = pd.to_numeric(deaths["Deaths"], errors="coerce").astype("Int64")
deaths = deaths[
    deaths["Drug/Alcohol Induced Cause"]
    == "Drug poisonings (overdose) Unintentional (X40-X44)"
]
deaths["fips"] = deaths["County Code"].astype(str).str.zfill(5)
deaths = deaths.drop(columns=["Year Code", "year", "County Code"])

In [20]:
deaths = deaths.rename(columns={"Year": "year"})
deaths = deaths.rename(columns={"Deaths": "deaths"})
deaths

,County,year,Drug/Alcohol Induced Cause,Drug/Alcohol Induced Cause Code,deaths,fips
1,"Baldwin County, AL",2006,Drug poisonings (overdose) Unintentional (X40-...,D1,11,01003
12,"Chilton County, AL",2006,Drug poisonings (overdose) Unintentional (X40-...,D1,13,01021
39,"Jefferson County, AL",2006,Drug poisonings (overdose) Unintentional (X40-...,D1,55,01073
55,"Mobile County, AL",2006,Drug poisonings (overdose) Unintentional (X40-...,D1,23,01097
60,"Montgomery County, AL",2006,Drug poisonings (overdose) Unintentional (X40-...,D1,12,01101
...,...,...,...,...,...,...
44778,"Waukesha County, WI",2015,Drug poisonings (overdose) Unintentional (X40-...,D1,34,55133
44784,"Winnebago County, WI",2015,Drug poisonings (overdose) Unintentional (X40-...,D1,22,55139
44794,"Fremont County, WY",2015,Drug poisonings (overdose) Unintentional (X40-...,D1,10,56013
44800,"Laramie County, WY",2015,Drug poisonings (overdose) Unintentional (X40-...,D1,13,56021


## Population

In [21]:
pop_1 = pd.read_csv("data/co-est00int-tot.csv", encoding="latin1")
pop_2 = pd.read_csv("data/co-est2020.csv", encoding="latin1")

In [22]:
p1_sub = pop_1[
    [
        "STATE",
        "COUNTY",
        "STNAME",
        "CTYNAME",
        "POPESTIMATE2006",
        "POPESTIMATE2007",
        "POPESTIMATE2008",
        "POPESTIMATE2009",
    ]
].copy()
p2_sub = pop_2[
    [
        "STATE",
        "COUNTY",
        "STNAME",
        "CTYNAME",
        "POPESTIMATE2010",
        "POPESTIMATE2011",
        "POPESTIMATE2012",
        "POPESTIMATE2013",
        "POPESTIMATE2014",
        "POPESTIMATE2015",
    ]
].copy()
pop_merged = p1_sub.merge(
    p2_sub, on=["STATE", "COUNTY", "STNAME", "CTYNAME"], how="inner"
)
pop_merged["fips"] = pop_merged["STATE"].astype(str).str.zfill(2) + pop_merged[
    "COUNTY"
].astype(str).str.zfill(3)

In [23]:
pop_merged

,STATE,COUNTY,STNAME,CTYNAME,POPESTIMATE2006,POPESTIMATE2007,POPESTIMATE2008,POPESTIMATE2009,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,fips
0,1,0,Alabama,Alabama,4628981,4672840,4718206,4757938,4785514,4799642,4816632,4831586,4843737,4854803,01000
1,1,1,Alabama,Autauga County,51328,52405,53277,54135,54761,55229,54970,54747,54922,54903,01001
2,1,3,Alabama,Baldwin County,168121,172404,175827,179406,183121,186579,190203,194978,199306,203101,01003
3,1,5,Alabama,Barbour County,27861,27757,27808,27657,27325,27344,27172,26946,26768,26300,01005
4,1,7,Alabama,Bibb County,22099,22438,22705,22941,22858,22736,22657,22510,22541,22553,01007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3183,56,37,Wyoming,Sweetwater County,39749,41470,42358,44133,43580,44000,45032,45189,44996,44780,56037
3184,56,39,Wyoming,Teton County,20014,20472,20988,21232,21298,21422,21643,22335,22801,23083,56039
3185,56,41,Wyoming,Uinta County,19709,20171,20613,21054,21090,20901,21008,20969,20835,20777,56041
3186,56,43,Wyoming,Washakie County,7979,8169,8229,8423,8531,8451,8410,8417,8277,8282,56043


In [24]:
year_cols = [c for c in pop_merged.columns if c.startswith("POPESTIMATE")]
pop_long = pop_merged.melt(
    id_vars=["fips", "STNAME", "CTYNAME"],
    value_vars=year_cols,
    var_name="pop_var",
    value_name="population",
)
pop_long["Year"] = pop_long["pop_var"].str.extract(r"(\d{4})").astype(int)
pop_long = pop_long.drop(columns=["pop_var"])
pop_long = pop_long.rename(columns={"STNAME": "state"})
pop_long = pop_long.rename(columns={"CTYNAME": "county"})
pop_long = pop_long.rename(columns={"Year": "year"})
pop_long["state"] = pop_long["state"].str.upper().str.strip()
pop_long["county"] = pop_long["county"].str.upper().str.strip()
pop_long

,fips,state,county,population,year
0,01000,ALABAMA,ALABAMA,4628981,2006
1,01001,ALABAMA,AUTAUGA COUNTY,51328,2006
2,01003,ALABAMA,BALDWIN COUNTY,168121,2006
3,01005,ALABAMA,BARBOUR COUNTY,27861,2006
4,01007,ALABAMA,BIBB COUNTY,22099,2006
...,...,...,...,...,...
31875,56037,WYOMING,SWEETWATER COUNTY,44780,2015
31876,56039,WYOMING,TETON COUNTY,23083,2015
31877,56041,WYOMING,UINTA COUNTY,20777,2015
31878,56043,WYOMING,WASHAKIE COUNTY,8282,2015


## Merging

In [25]:
deaths

,County,year,Drug/Alcohol Induced Cause,Drug/Alcohol Induced Cause Code,deaths,fips
1,"Baldwin County, AL",2006,Drug poisonings (overdose) Unintentional (X40-...,D1,11,01003
12,"Chilton County, AL",2006,Drug poisonings (overdose) Unintentional (X40-...,D1,13,01021
39,"Jefferson County, AL",2006,Drug poisonings (overdose) Unintentional (X40-...,D1,55,01073
55,"Mobile County, AL",2006,Drug poisonings (overdose) Unintentional (X40-...,D1,23,01097
60,"Montgomery County, AL",2006,Drug poisonings (overdose) Unintentional (X40-...,D1,12,01101
...,...,...,...,...,...,...
44778,"Waukesha County, WI",2015,Drug poisonings (overdose) Unintentional (X40-...,D1,34,55133
44784,"Winnebago County, WI",2015,Drug poisonings (overdose) Unintentional (X40-...,D1,22,55139
44794,"Fremont County, WY",2015,Drug poisonings (overdose) Unintentional (X40-...,D1,10,56013
44800,"Laramie County, WY",2015,Drug poisonings (overdose) Unintentional (X40-...,D1,13,56021


In [26]:
pop_long

,fips,state,county,population,year
0,01000,ALABAMA,ALABAMA,4628981,2006
1,01001,ALABAMA,AUTAUGA COUNTY,51328,2006
2,01003,ALABAMA,BALDWIN COUNTY,168121,2006
3,01005,ALABAMA,BARBOUR COUNTY,27861,2006
4,01007,ALABAMA,BIBB COUNTY,22099,2006
...,...,...,...,...,...
31875,56037,WYOMING,SWEETWATER COUNTY,44780,2015
31876,56039,WYOMING,TETON COUNTY,23083,2015
31877,56041,WYOMING,UINTA COUNTY,20777,2015
31878,56043,WYOMING,WASHAKIE COUNTY,8282,2015


In [27]:
prescriptions_merged

,buyer_state,buyer_county,year,calc_base_wt_in_gm,fips
0,AK,ANCHORAGE,2006,143823.375649,02020
1,AK,ANCHORAGE,2007,145660.162020,02020
2,AK,ANCHORAGE,2008,151111.681778,02020
3,AK,ANCHORAGE,2009,154228.894910,02020
4,AK,ANCHORAGE,2010,153320.451041,02020
...,...,...,...,...,...
42117,WY,WESTON,2015,2725.641288,56045
42118,WY,WESTON,2016,2284.998660,56045
42119,WY,WESTON,2017,1893.343744,56045
42120,WY,WESTON,2018,1701.088712,56045


In [28]:
presc_deaths = pd.merge(
    prescriptions_merged,
    deaths[["fips", "County", "year", "deaths"]],
    on=["fips", "year"],
    how="outer",
)

opioids = pd.merge(
    presc_deaths,
    pop_long[["fips", "year", "county", "state", "population"]],
    on=["fips", "year"],
    how="left",
)

opioids = opioids[
    [
        "fips",
        "buyer_county",
        "County",
        "county",
        "state",
        "buyer_state",
        "year",
        "calc_base_wt_in_gm",
        "deaths",
        "population",
    ]
]

print(opioids.shape)
opioids.sample(20)

(42167, 10)


,fips,buyer_county,County,county,state,buyer_state,year,calc_base_wt_in_gm,deaths,population
29022,40061,HASKELL,NaN,HASKELL COUNTY,OKLAHOMA,OK,2015,10585.792928,<NA>,12749.0
32610,47013,CAMPBELL,NaN,CAMPBELL COUNTY,TENNESSEE,TN,2009,52176.279584,<NA>,40790.0
21849,30027,FERGUS,NaN,FERGUS COUNTY,MONTANA,MT,2015,4521.793254,<NA>,11315.0
1517,05021,CLAY,NaN,NaN,NaN,AR,2016,8568.240368,<NA>,NaN
26411,37155,ROBESON,NaN,NaN,NaN,NC,2018,86678.421408,<NA>,NaN
41499,55117,SHEBOYGAN,NaN,NaN,NaN,WI,2019,18049.809514,<NA>,NaN
14051,21107,HOPKINS,NaN,HOPKINS COUNTY,KENTUCKY,KY,2007,30700.102035,<NA>,46834.0
27519,39023,CLARK,"Clark County, OH",CLARK COUNTY,OHIO,OH,2010,63874.728150,20,138272.0
9524,18043,FLOYD,NaN,NaN,NaN,IN,2016,59297.465941,<NA>,NaN
29646,40151,WOODS,NaN,WOODS COUNTY,OKLAHOMA,OK,2009,3060.329455,<NA>,8708.0


## Preparation

In [29]:
opioids_clean = opioids.copy()
avg_pop = opioids_clean.groupby("fips")["population"].mean()
big_fips = avg_pop[avg_pop >= 350000].index
opioids_clean = opioids_clean[opioids_clean["fips"].isin(big_fips)]
opioids_clean.shape

(2624, 10)

In [30]:
opioids_clean

,fips,buyer_county,County,county,state,buyer_state,year,calc_base_wt_in_gm,deaths,population
504,01073,JEFFERSON,"Jefferson County, AL",JEFFERSON COUNTY,ALABAMA,AL,2006,399232.868993,55,655893.0
505,01073,JEFFERSON,"Jefferson County, AL",JEFFERSON COUNTY,ALABAMA,AL,2007,420555.537846,67,655163.0
506,01073,JEFFERSON,"Jefferson County, AL",JEFFERSON COUNTY,ALABAMA,AL,2008,440411.156173,63,656510.0
507,01073,JEFFERSON,"Jefferson County, AL",JEFFERSON COUNTY,ALABAMA,AL,2009,454896.396006,80,658441.0
508,01073,JEFFERSON,"Jefferson County, AL",JEFFERSON COUNTY,ALABAMA,AL,2010,486677.671546,53,658230.0
...,...,...,...,...,...,...,...,...,...,...
41607,55133,WAUKESHA,"Waukesha County, WI",WAUKESHA COUNTY,WISCONSIN,WI,2015,200375.123762,34,396377.0
41608,55133,WAUKESHA,NaN,NaN,NaN,WI,2016,192636.391662,<NA>,NaN
41609,55133,WAUKESHA,NaN,NaN,NaN,WI,2017,154116.663589,<NA>,NaN
41610,55133,WAUKESHA,NaN,NaN,NaN,WI,2018,126187.189934,<NA>,NaN


In [31]:
opioids_clean["deaths"] = pd.to_numeric(
    opioids_clean["deaths"], errors="coerce"
).astype("Int64")
opioids_clean["population"] = opioids_clean["population"].astype("Int64")
opioids_clean.dtypes

fips                   object
buyer_county           object
County                 object
county                 object
state                  object
buyer_state            object
year                    Int64
calc_base_wt_in_gm    float64
deaths                  Int64
population              Int64
dtype: object

In [32]:
opioids_clean["policy_state"] = (opioids_clean["state"] == "FLORIDA").astype(int)
opioids_clean["post"] = (opioids_clean["year"] >= 2010).astype(int)
opioids_clean["prescriptions_per_1000"] = (
    opioids_clean["calc_base_wt_in_gm"] / opioids_clean["population"] * 1000
)
opioids_clean["overdose_per_100k"] = (
    opioids_clean["deaths"] / opioids_clean["population"] * 100000
)
opioids_clean["relative_year"] = opioids_clean["year"] - 2010

In [33]:
opioids_clean.dtypes

fips                       object
buyer_county               object
County                     object
county                     object
state                      object
buyer_state                object
year                        Int64
calc_base_wt_in_gm        float64
deaths                      Int64
population                  Int64
policy_state                int64
post                        int64
prescriptions_per_1000    Float64
overdose_per_100k         Float64
relative_year               Int64
dtype: object

In [ ]:
# opioids_clean.to_csv("opioids_clean.csv")